In [1]:
import requests as re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import altair as alt
import lxml

Scrape fantrax standings

In [14]:
def buildDF(content):
    table = content.find("ultimate-table")
    names =  table.find("aside").find_all("td")
    rankings = []
    name_list = []
    for td in names:
        rank_and_name = td.text.split("  ")
        rank = rank_and_name[0]
        name = rank_and_name[1]
        rankings.append(rank)
        name_list.append(name)
    data = table.find("table")
    df = pd.read_html(str(data))[0]
    df.columns = ["W","D","L","Points","Win%","WW","FPtsF","FPtsA","Streak"]
    df["team"] = name_list
    df["rank"] = rankings
    df["gw"] = gw
    return df

In [22]:
df_list = []

for i in range(38):

  # open web browser for each gameweek
  gw = i + 1
  url= f"https://www.fantrax.com/fantasy/league/m2cauar6l5yu7qdj/standings;view=REGULAR_SEASON;timeframeType=BY_PERIOD;timeStartType=FROM_SEASON_START;period={gw}"
  driver = webdriver.Chrome()
  driver.get(url)

  # wait for the table to load
  try:
    elem = WebDriverWait(driver, 30).until(
      EC.presence_of_element_located((By.TAG_NAME, "ultimate-table")) #This is a dummy element
    )
  finally:
    html = driver.page_source
    soup=BeautifulSoup(html,"html.parser")
    content = soup.find("div", {"class":"league-standings-table"})

  df = buildDF(content)

  df_list.append(df)

In [23]:
concat_df = pd.concat(df_list)

In [37]:
concat_df['team'] = concat_df['team'].str.strip()

In [39]:
concat_df['rank'] = concat_df['rank'].str.strip().astype(int)

In [109]:
data = concat_df#[concat_df.team != "RATABE" ]

chart = alt.Chart(data).mark_line(point=True).encode(
    x = alt.X('gw', scale=alt.Scale(domain=[0, 38]), title="Gameweek"),
    y=alt.Y('rank', aggregate={'argmax': 'gw'}, scale=alt.Scale(domain=[8,1]), title="Rank"),
    color=alt.Color("team", legend=None),
).transform_window(
    rank="rank()",
    sort=[alt.SortField("rank", order="ascending")],
    groupby=["gw"]
).properties(
    title="Fake Internet Soccer XII standings by gameweek",
    width=700,
    height=350,
)

labels = alt.Chart(data).mark_text(
    align='left', dx=5
).encode(
    x = alt.X('max(gw)', scale=alt.Scale(domain=[0, 38])),
    y=alt.Y('rank', aggregate={'argmax': 'gw'}, scale=alt.Scale(domain=[8,1])),
    text='team:N',
    color='team:N',
).transform_window(
    rank="rank()",
    sort=[alt.SortField("rank", order="ascending")],
    groupby=["gw"]
)

chart + labels

alt.LayerChart(...)

In [110]:
chart = alt.Chart(concat_df).mark_line(point=True).encode(
    x = alt.X('gw', scale=alt.Scale(domain=[0, 38]), title="Gameweek"),
    y=alt.Y("Points", title="Points"),
    color=alt.Color("team", legend=None),
    tooltip=["team"]
).transform_window(
    rank="rank()",
    sort=[alt.SortField("rank", order="descending")],
    groupby=["gw"]
).properties(
    title="Fake Internet Soccer XII standings by gameweek",
    width=700,
    height=350,
)

labels = alt.Chart(concat_df).mark_text(
    align='left', dx=5
).encode(
    x = alt.X('max(gw)', scale=alt.Scale(domain=[0, 38])),
    y=alt.Y('Points', aggregate={'argmax': 'gw'}),
    text='team:N',
    color='team:N',
).transform_window(
    rank="rank()",
    sort=[alt.SortField("rank", order="descending")],
    groupby=["gw"]
)

chart + labels

alt.LayerChart(...)

In [115]:
concat_df[concat_df.team == "Seanhampton FC"]

,W,D,L,Points,Win%,WW,FPtsF,FPtsA,Streak,team,rank,gw
3,1,0,0,3,1.000,6,58.4,52.3,1 (W),Seanhampton FC,4,1
4,1,0,1,3,0.500,6,86.2,97.3,1 (L),Seanhampton FC,5,2
5,1,0,2,3,0.333,6,123.9,146.6,2 (L),Seanhampton FC,6,3
7,1,0,3,3,0.250,6,175.7,201.7,3 (L),Seanhampton FC,8,4
7,1,0,4,3,0.200,6,211.6,255.9,4 (L),Seanhampton FC,8,5
7,1,0,5,3,0.167,6,252.2,332.6,5 (L),Seanhampton FC,8,6
7,1,1,5,4,0.214,6,252.2,332.6,6 (Wnl),Seanhampton FC,8,7
7,1,1,6,4,0.188,6,287.7,379.0,7 (Wnl),Seanhampton FC,8,8
7,1,1,7,4,0.167,6,328.4,424.5,8 (Wnl),Seanhampton FC,8,9
7,1,1,8,4,0.150,6,368.0,497.0,9 (Wnl),Seanhampton FC,8,10
